# Computed results

## Imports

### Install python needs

In [1]:
%pip install git+https://github.com/EEmilieNN/droplet_impact.git

  Cloning https://github.com/EEmilieNN/droplet_impact.git to /private/var/folders/f2/qc2dpyh901z6jbdzg2w6lt0m0000gn/T/pip-req-build-hu4rydce
  Running command git clone --filter=blob:none --quiet https://github.com/EEmilieNN/droplet_impact.git /private/var/folders/f2/qc2dpyh901z6jbdzg2w6lt0m0000gn/T/pip-req-build-hu4rydce
  Resolved https://github.com/EEmilieNN/droplet_impact.git to commit adb2ebb21f16ecaa967cfb4e305e8af9e0c4a1ee
  Preparing metadata (setup.py) ... done
  Created wheel for droplet-impact: filename=droplet_impact-0.1.0-py3-none-any.whl size=6499084 sha256=c6bf00e7d6fa38011ad50eed994aa0cf55ff8582b0cee63b93a7a56998534360
  Stored in directory: /private/var/folders/f2/qc2dpyh901z6jbdzg2w6lt0m0000gn/T/pip-ephem-wheel-cache-5wwaim5t/wheels/92/3c/59/d019641b5398ed9b1e39ec67e926c1f4c7a0f0e868ac9b04f6
Successfully built droplet-impact
Note: you may need to restart the kernel to use updated packages.


### Import packages

In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from scipy.interpolate import RegularGridInterpolator, RBFInterpolator
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from droplet_impact import utils
import joblib

## Run the simulation for the values we need (already done)

It will calculate the impact speed for a lot of values and store it in impact_speed[i,j,k,l]

In [ ]:
import droplet_impact.config as cfg

# Define the parameter ranges
V_blade_vals =np.concatenate([np.linspace(60, 70, 15),np.linspace(70, 72.5, 4),np.linspace(72.5, 75, 4),np.linspace(75, 77.5, 4),np.linspace(77.5, 80, 4),np.linspace(80, 90, 15),np.linspace(90, 100, 15)])  # m/s
R_vals = np.linspace(0.0001, 0.002, 50)         # m (converted from mm)
Rc_vals = np.concatenate([np.linspace(0.01, 0.2, 30),[0.5,1]])            # m (converted from cm)
n_vals = np.linspace(1.0, 2.0, 10)


# Meshgrid is not necessary unless you want to loop with indices
impact_speeds = np.empty((len(V_blade_vals), len(R_vals), len(Rc_vals), len(n_vals)))

for i, V in enumerate(V_blade_vals):
    for j, R in enumerate(R_vals):
        for k, Rc in enumerate(Rc_vals):
            for l, n in enumerate(n_vals):
                # Set the configuration parameters
                cfg.V_blade = V
                cfg.R = R
                cfg.Rc_alpha = Rc
                cfg.n = n
                cfg.vx0 = -utils.v_terminal(cfg.R*1e3)
                cfg.vx_blade0 = cfg.V_blade
                initial_conditions = [cfg.x0, cfg.vx0, cfg.R, cfg.va0, -1, cfg.vx_blade0]
                # Solve your model using solve_ivp
                impact_speeds[i, j, k, l] = utils.impact_speed_vertical(initial_conditions, time_span = (0,0.1), time_steps = np.linspace(0, 0.1, 1000000), nose_radius = cfg.Rc_alpha, n = cfg.n, initial_radius = cfg.R, blade_speed = cfg.V_blade)

In [ ]:
### Save the results to a file

# np.savez_compressed('impact_speeds.npz', impact_speeds=impact_speeds, V_blade_vals=V_blade_vals, R_vals=R_vals, Rc_vals=Rc_vals, n_vals=n_vals)

## Interpolate the values

We take the different files we have to combine them into a big file

In [2]:
import numpy as np
from collections import defaultdict

# Define the parameter ranges
V_blade_vals =np.concatenate([np.linspace(60, 70, 15),np.linspace(70, 72.5, 4),np.linspace(72.5, 75, 4),np.linspace(75, 77.5, 4),np.linspace(77.5, 80, 4),np.linspace(80, 90, 15),np.linspace(90, 100, 15)])  # m/s
R_vals = np.linspace(0.0001, 0.002, 50)         # m (converted from mm)
Rc_vals = np.concatenate([np.linspace(0.01, 0.2, 30),[0.5,1]])            # m (converted from cm)
n_vals = np.linspace(1.0, 2.0, 10)

file_paths = [
    "raw_data/impact_speeds_60_70.npz",
    "raw_data/impact_speeds_70_72.npz",
    "raw_data/impact_speeds_72_75.npz",
    "raw_data/impact_speeds_75_77.npz",
    "raw_data/impact_speeds_77_80.npz",
    "raw_data/impact_speeds_80_90.npz",
    "raw_data/impact_speeds_90_100.npz"
]

v_blade_ranges = [
    (60, 70, 15),
    (70, 72.5, 4),
    (72.5, 75, 4),
    (75, 77.5, 4),
    (77.5, 80, 4),
    (80, 90, 15),
    (90, 100, 15)
]

unique_impact_speeds = defaultdict(dict)

for file_path, (v_start, v_end, step) in zip(file_paths, v_blade_ranges):
    data = np.load(file_path)
    impact_speeds = data['impact_speeds']

    v_blade_values = np.linspace(v_start, v_end, step)

    for v_blade, speed in zip(v_blade_values, impact_speeds):
        if v_blade not in unique_impact_speeds:
            unique_impact_speeds[v_blade] = speed

sorted_v_blade = sorted(unique_impact_speeds.keys())
combined_impact_speeds = np.array([unique_impact_speeds[v] for v in sorted_v_blade])

combined_file_path = "output_data/combined_impact_speeds.npz"
np.savez(combined_file_path, impact_speeds=combined_impact_speeds, v_blade=sorted_v_blade)



We interpolate the values into an interpolator that we export for future uses

In [3]:
# Create a RegularGridInterpolator for the impact speeds
from scipy.interpolate import RegularGridInterpolator

data = np.load("output_data/combined_impact_speeds.npz")
impact_speeds = data['impact_speeds']
V_blade_vals = data['v_blade']

interpolator = RegularGridInterpolator(
    (V_blade_vals, R_vals, Rc_vals, n_vals),
    impact_speeds,
    bounds_error=False,
    fill_value=None
)

joblib.dump(interpolator, 'output_data/impact_speed_interpolator.pkl')

['output_data/impact_speed_interpolator.pkl']

## Test if the interpolation is correct

We calculate the min, max and average error for several points to see how far we are with the interpolation

In [4]:
l =[]
for vvv in [60,70,80,90,100]:
    for rrr in [0.2e-3, 0.49e-3, 0.76e-3,1e-3]:
        for rccc in [0.01, 0.05, 0.1, 0.2]:
            for nn in [1.0, 1.2, 1.4, 1.6, 1.8, 2.0]:
                v = utils.get_impact_speed(vvv, rrr, rccc, nn)
                theory_v = utils.impact_speed_vertical([0, -utils.v_terminal(rrr*1e3), rrr, 0, -1, vvv], time_span=(0, 0.1), time_steps=np.linspace(0, 0.1, 1000000), nose_radius=rccc, n=nn, initial_radius=rrr, blade_speed=vvv)
                l.append(abs(v-theory_v))

print(f"\nAverage error: {np.mean(l):.6f} m/s")
print(f"Max error: {np.max(l):.6f} m/s")
print(f"Min error: {np.min(l):.6f} m/s")

/Users/emilien/miniconda3/lib/python3.10/site-packages/droplet_impact/physics_model.py:165: RuntimeWarning: overflow encountered in scalar power
  C_stat = pow(C_D_sphere,b/a) * pow(cfg.C_D_disk,1 - b/a)
/Users/emilien/miniconda3/lib/python3.10/site-packages/droplet_impact/physics_model.py:165: RuntimeWarning: invalid value encountered in scalar multiply
  C_stat = pow(C_D_sphere,b/a) * pow(cfg.C_D_disk,1 - b/a)


KeyboardInterrupt: 